In [1]:
import pandas as pd
import numpy as np
import time
import os
from os import listdir
from os.path import isfile, join, basename

import sys

from lag_features import *
from other_functions import *

import datetime
from sklearn.preprocessing import LabelEncoder

In [2]:
DIR = '/Users/carlosperezricardo/Desktop/TFM'

#[10397, 13930, 11298, 11292, 12892]
AIRPORT = 10397
years_to_load = ['2016','2017','2018','2019']

In [3]:
folder = os.path.join(DIR, 'datasets')
files = [f for f in listdir(folder) if isfile(join(folder, f))]
files = sorted(files)

In [4]:
df = pd.DataFrame()
for file in files:
    loading = []
    if '.zip' in file:
        for year in years_to_load:
            loading.append(year in file)
        output = any(loading)
        if output:
            add_df = pd.read_csv(os.path.join(DIR,'datasets',file), parse_dates=['FL_DATE'])
            add_df = add_df[add_df.DEST_AIRPORT_ID == AIRPORT]
            df = pd.concat([df, add_df], axis=0)
df.reset_index(drop=True, inplace=True)

/Users/carlosperezricardo/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (23) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
df.head()

,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,...,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 29
0,2016-01-01,AA,19805,AA,N3KVAA,1178,11298,1129804,30194,DFW,...,24.0,-3.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
1,2016-01-01,AA,19805,AA,N3AUAA,1427,11298,1129804,30194,DFW,...,1205.0,-10.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2,2016-01-01,AA,19805,AA,N479AA,1442,11298,1129804,30194,DFW,...,1439.0,42.0,0.0,NaN,0.0,1.0,0.0,0.0,41.0,NaN
3,2016-01-01,AS,19930,AS,N477AS,746,14747,1474703,30559,SEA,...,1736.0,-2.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
4,2016-01-01,AS,19930,AS,N462AS,750,14747,1474703,30559,SEA,...,2354.0,-4.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN


In [6]:
df['DEST'].value_counts()

ATL    1500685
Name: DEST, dtype: int64

In [7]:
ini_date = datetime.datetime(2017, 1, 1)

train_ini_date = datetime.datetime(2018, 1, 1)
train_fin_date = datetime.datetime(2019, 9, 1)

test_ini_date = datetime.datetime(2019, 9, 1)
test_fin_date = datetime.datetime(2019, 9, 30)

In [8]:
df = df[(df.FL_DATE > ini_date) & (df.FL_DATE <= test_fin_date)]

In [9]:
df.shape

(1051742, 30)

In [10]:
# Feature Generation 
df = date_features(df, 'FL_DATE')

In [11]:
# Add weather
WEATHER_DIR = '/Users/carlosperezricardo/Desktop/TFM/weather'
weather_df = pd.read_csv(os.path.join(WEATHER_DIR, 'weather.csv'))

In [12]:
iata_code = list(df['DEST'].unique())[0]
iata_code

'ATL'

In [13]:
airports_df = pd.read_csv(os.path.join('/Users/carlosperezricardo/Desktop/TFM/airlines_airports', 'airports.csv'))

airports_code = pd.read_excel(os.path.join('/Users/carlosperezricardo/Desktop/TFM/airlines_airports', 'US_airports.xlsx'), sheet_name='Airports')
airports_code.columns = airports_code.iloc[0]
airports_code = airports_code.iloc[1:]
    
states = pd.read_excel(os.path.join('/Users/carlosperezricardo/Desktop/TFM/airlines_airports', 'US_airports.xlsx'), sheet_name='States')
states.columns = ['state_name','state_acronym']

airports = pd.merge(airports_code, states, left_on='City', right_on='state_name', how='left')

airports[['state_name','state_acronym']] = airports[['state_name','state_acronym']].ffill(axis = 0)
airports = airports[~airports['FAA'].isnull()]

In [14]:
icao_code = list(airports[airports.IATA == iata_code]['ICAO'])[0]
icao_code

'KATL'

In [15]:
weather_df = weather_df[weather_df.ICAO == icao_code]
weather_df['Day'] = pd.to_datetime(weather_df['Day'])
weather_df.head()

,Unnamed: 0,Day,Temperature_Max,Temperature_Avg,Temperature_Min,Dew Point_Max,Dew Point_Avg,Dew Point_Min,Humidity_Max,Humidity_Avg,Humidity_Min,Wind Speed_Max,Wind Speed_Avg,Wind Speed_Min,Pressure_Max,Pressure_Avg,Pressure_Min,Precipitation,ICAO
85792,85792,2019-01-01,59,53.8,49,53,49.0,44,93,84.3,69,9.0,5.4,0.0,29.1,29.1,28.9,0.00,KATL
85793,85793,2019-01-02,55,53.1,50,53,50.6,49,96,91.3,86,14.0,4.6,0.0,29.0,28.9,28.9,0.60,KATL
85794,85794,2019-01-03,66,55.8,45,58,50.7,37,97,83.8,58,25.0,10.9,3.0,28.9,28.7,28.6,1.00,KATL
85795,85795,2019-01-04,59,49.0,44,38,36.4,34,79,63.3,41,21.0,14.2,7.0,29.0,28.9,28.7,0.56,KATL
85796,85796,2019-01-05,69,54.5,43,41,38.2,36,80,56.7,33,10.0,6.6,0.0,29.1,29.1,29.0,0.00,KATL


In [16]:
df = pd.merge(df, weather_df, left_on='FL_DATE', right_on='Day', how='left')

In [17]:
df.head()

,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,...,Humidity_Avg,Humidity_Min,Wind Speed_Max,Wind Speed_Avg,Wind Speed_Min,Pressure_Max,Pressure_Avg,Pressure_Min,Precipitation,ICAO
0,2017-01-02,AA,19805,AA,N3CLAA,1249,12892,1289204,32575,LAX,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-01-02,DL,19790,DL,N920DE,2392,11267,1126702,31267,DAY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-01-02,DL,19790,DL,N995AT,2393,12217,1221702,30255,HSV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-01-02,DL,19790,DL,N950DN,2396,14635,1463502,31714,RSW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-01-02,DL,19790,DL,N672DL,2401,13198,1319801,33198,MCI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
df.groupby('TAIL_NUM')['ARR_DELAY'].apply(lambda x: x.shift(1))

0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
           ... 
1051737     3.0
1051738    32.0
1051739    -2.0
1051740   -18.0
1051741   -13.0
Name: ARR_DELAY, Length: 1051742, dtype: float64

In [19]:
df.shape

(1051742, 54)

In [20]:
df.head()

,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,...,Humidity_Avg,Humidity_Min,Wind Speed_Max,Wind Speed_Avg,Wind Speed_Min,Pressure_Max,Pressure_Avg,Pressure_Min,Precipitation,ICAO
0,2017-01-02,AA,19805,AA,N3CLAA,1249,12892,1289204,32575,LAX,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-01-02,DL,19790,DL,N920DE,2392,11267,1126702,31267,DAY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-01-02,DL,19790,DL,N995AT,2393,12217,1221702,30255,HSV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-01-02,DL,19790,DL,N950DN,2396,14635,1463502,31714,RSW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-01-02,DL,19790,DL,N672DL,2401,13198,1319801,33198,MCI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
df.columns

Index(['FL_DATE', 'OP_UNIQUE_CARRIER', 'OP_CARRIER_AIRLINE_ID', 'OP_CARRIER',
       'TAIL_NUM', 'OP_CARRIER_FL_NUM', 'ORIGIN_AIRPORT_ID',
       'ORIGIN_AIRPORT_SEQ_ID', 'ORIGIN_CITY_MARKET_ID', 'ORIGIN',
       'ORIGIN_CITY_NAME', 'ORIGIN_STATE_ABR', 'DEST_AIRPORT_ID',
       'DEST_AIRPORT_SEQ_ID', 'DEST_CITY_MARKET_ID', 'DEST', 'DEST_CITY_NAME',
       'DEST_STATE_ABR', 'DEP_TIME', 'DEP_DELAY', 'ARR_TIME', 'ARR_DELAY',
       'CANCELLED', 'CANCELLATION_CODE', 'DIVERTED', 'CARRIER_DELAY',
       'WEATHER_DELAY', 'NAS_DELAY', 'LATE_AIRCRAFT_DELAY', 'Unnamed: 29',
       'month', 'day', 'year', 'year_month', 'weekday', 'Unnamed: 0', 'Day',
       'Temperature_Max', 'Temperature_Avg', 'Temperature_Min',
       'Dew Point_Max', 'Dew Point_Avg', 'Dew Point_Min', 'Humidity_Max',
       'Humidity_Avg', 'Humidity_Min', 'Wind Speed_Max', 'Wind Speed_Avg',
       'Wind Speed_Min', 'Pressure_Max', 'Pressure_Avg', 'Pressure_Min',
       'Precipitation', 'ICAO'],
      dtype='object')

In [22]:
train_df = df[(df.FL_DATE > datetime.datetime(2018, 1, 1)) & (df.FL_DATE <= datetime.datetime(2019, 9, 1))]
test_df = df[(df.FL_DATE > datetime.datetime(2019, 9, 1)) & (df.FL_DATE <= datetime.datetime(2019, 9, 30))]

In [23]:
print(train_df.shape)
print(test_df.shape)

(655979, 54)
(31168, 54)


In [24]:
df_ = df.copy(deep=True)

In [25]:
df_ = df_[(df_.CANCELLED != 1) & (df_.DIVERTED != 1)]

In [26]:
# Lag features by: TAIL_NUM
calculations = {}
calculations['calc1'] = {'gb_list':['TAIL_NUM','FL_DATE'],'target':'ARR_DELAY','shifts':[15], 'windows':[3,5,10], 'funs':['mean']}
#calculations['calc1'] = {'gb_list':['TAIL_NUM','FL_DATE'],'target':'ARR_DELAY','shifts':[30,45,60], 'windows':[10], 'funs':['mean','std']}
#calculations['calc2'] = {'gb_list':['TAIL_NUM','FL_DATE'],'target':'ARR_DELAY','shifts':[365], 'windows':[5,10], 'funs':['mean']}

df_ = apply_calc(df_, calculations)

Generating ARR_DELAY_TAIL_NUM_FL_DATE_s15_r3_mean
Generating ARR_DELAY_TAIL_NUM_FL_DATE_s15_r5_mean
Generating ARR_DELAY_TAIL_NUM_FL_DATE_s15_r10_mean


In [27]:
# Lag features by: DEST_AIRPORT_ID
calculations = {}
calculations['calc3'] = {'gb_list':['DEST_AIRPORT_ID','FL_DATE'],'target':'ARR_DELAY','shifts':[15], 'windows':[5,10,30], 'funs':['mean']}
calculations['calc4'] = {'gb_list':['DEST_AIRPORT_ID','FL_DATE'],'target':'ARR_DELAY','shifts':[15,30], 'windows':[15,30], 'funs':['median','std']}
calculations['calc5'] = {'gb_list':['DEST_AIRPORT_ID','FL_DATE'],'target':'ARR_DELAY','shifts':[365], 'windows':[10], 'funs':['mean','std']}

df_ = apply_calc(df_, calculations)

Generating ARR_DELAY_DEST_AIRPORT_ID_FL_DATE_s15_r5_mean
Generating ARR_DELAY_DEST_AIRPORT_ID_FL_DATE_s15_r10_mean
Generating ARR_DELAY_DEST_AIRPORT_ID_FL_DATE_s15_r30_mean
Generating ARR_DELAY_DEST_AIRPORT_ID_FL_DATE_s15_r15_median
Generating ARR_DELAY_DEST_AIRPORT_ID_FL_DATE_s30_r15_median
Generating ARR_DELAY_DEST_AIRPORT_ID_FL_DATE_s15_r30_median
Generating ARR_DELAY_DEST_AIRPORT_ID_FL_DATE_s30_r30_median
Generating ARR_DELAY_DEST_AIRPORT_ID_FL_DATE_s15_r15_std
Generating ARR_DELAY_DEST_AIRPORT_ID_FL_DATE_s30_r15_std
Generating ARR_DELAY_DEST_AIRPORT_ID_FL_DATE_s15_r30_std
Generating ARR_DELAY_DEST_AIRPORT_ID_FL_DATE_s30_r30_std
Generating ARR_DELAY_DEST_AIRPORT_ID_FL_DATE_s365_r10_mean
Generating ARR_DELAY_DEST_AIRPORT_ID_FL_DATE_s365_r10_std


In [28]:
# Lag features by: ORIGIN_AIRPORT_ID
calculations = {}
calculations['calc6'] = {'gb_list':['ORIGIN_AIRPORT_ID','FL_DATE'],'target':'DEP_DELAY','shifts':[15,30,45], 'windows':[5,10,30], 'funs':['mean']}
calculations['calc7'] = {'gb_list':['DEST_AIRPORT_ID','FL_DATE'],'target':'DEP_DELAY','shifts':[15,30], 'windows':[15,30], 'funs':['median','std']}
calculations['calc8'] = {'gb_list':['ORIGIN_AIRPORT_ID','FL_DATE'],'target':'DEP_DELAY','shifts':[365], 'windows':[10], 'funs':['mean']}

df_ = apply_calc(df_, calculations)

Generating DEP_DELAY_ORIGIN_AIRPORT_ID_FL_DATE_s15_r5_mean
Generating DEP_DELAY_ORIGIN_AIRPORT_ID_FL_DATE_s30_r5_mean
Generating DEP_DELAY_ORIGIN_AIRPORT_ID_FL_DATE_s45_r5_mean
Generating DEP_DELAY_ORIGIN_AIRPORT_ID_FL_DATE_s15_r10_mean
Generating DEP_DELAY_ORIGIN_AIRPORT_ID_FL_DATE_s30_r10_mean
Generating DEP_DELAY_ORIGIN_AIRPORT_ID_FL_DATE_s45_r10_mean
Generating DEP_DELAY_ORIGIN_AIRPORT_ID_FL_DATE_s15_r30_mean
Generating DEP_DELAY_ORIGIN_AIRPORT_ID_FL_DATE_s30_r30_mean
Generating DEP_DELAY_ORIGIN_AIRPORT_ID_FL_DATE_s45_r30_mean
Generating DEP_DELAY_DEST_AIRPORT_ID_FL_DATE_s15_r15_median
Generating DEP_DELAY_DEST_AIRPORT_ID_FL_DATE_s30_r15_median
Generating DEP_DELAY_DEST_AIRPORT_ID_FL_DATE_s15_r30_median
Generating DEP_DELAY_DEST_AIRPORT_ID_FL_DATE_s30_r30_median
Generating DEP_DELAY_DEST_AIRPORT_ID_FL_DATE_s15_r15_std
Generating DEP_DELAY_DEST_AIRPORT_ID_FL_DATE_s30_r15_std
Generating DEP_DELAY_DEST_AIRPORT_ID_FL_DATE_s15_r30_std
Generating DEP_DELAY_DEST_AIRPORT_ID_FL_DATE_s30_r30

In [29]:
# Lag features by: OP_CARRIER_AIRLINE_ID
calculations = {}
calculations['calc3'] = {'gb_list':['OP_CARRIER_AIRLINE_ID','FL_DATE'],'target':'ARR_DELAY','shifts':[15,30], 'windows':[10,30], 'funs':['mean']}
calculations['calc4'] = {'gb_list':['OP_CARRIER_AIRLINE_ID','FL_DATE'],'target':'ARR_DELAY','shifts':[30], 'windows':[10], 'funs':['median','std']}
calculations['calc5'] = {'gb_list':['OP_CARRIER_AIRLINE_ID','FL_DATE'],'target':'ARR_DELAY','shifts':[365], 'windows':[10], 'funs':['mean','std']}

calculations['calc3'] = {'gb_list':['OP_CARRIER_AIRLINE_ID','FL_DATE'],'target':'DEP_DELAY','shifts':[15,30], 'windows':[10,30], 'funs':['mean']}
calculations['calc4'] = {'gb_list':['OP_CARRIER_AIRLINE_ID','FL_DATE'],'target':'DEP_DELAY','shifts':[30], 'windows':[10], 'funs':['median','std']}
calculations['calc5'] = {'gb_list':['OP_CARRIER_AIRLINE_ID','FL_DATE'],'target':'DEP_DELAY','shifts':[365], 'windows':[10], 'funs':['mean','std']}

df_ = apply_calc(df_, calculations)

Generating DEP_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s15_r10_mean
Generating DEP_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s30_r10_mean
Generating DEP_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s15_r30_mean
Generating DEP_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s30_r30_mean
Generating DEP_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s30_r10_median
Generating DEP_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s30_r10_std
Generating DEP_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s365_r10_mean
Generating DEP_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s365_r10_std


In [30]:
# Convert times 
# [ARR_TIME, DEP_TIME]

df_['DEP_TIME'] = df_['DEP_TIME']/100
df_['ARR_TIME'] = df_['ARR_TIME']/100

df_['DEP_TIME'] = df_['DEP_TIME'].astype(int)
df_['ARR_TIME'] = df_['ARR_TIME'].astype(int)

In [31]:
df_['DEP_n_flights_per_hour'] = df_.groupby(['FL_DATE','DEP_TIME'])['ARR_TIME'].transform( lambda x: x.count())
df_['ARR_n_flights_per_hour'] = df_.groupby(['FL_DATE','ARR_TIME'])['DEP_TIME'].transform( lambda x: x.count())

df_['n_flights_per_hour'] = df_['DEP_n_flights_per_hour'] + df_['ARR_n_flights_per_hour'] 

In [32]:
df_['DEP_n_flights_cumsum'] = df_.groupby(['FL_DATE'])['DEP_n_flights_per_hour'].transform( lambda x: x.cumsum())
df_['ARR_n_flights_cumsum'] = df_.groupby(['FL_DATE'])['ARR_n_flights_per_hour'].transform( lambda x: x.cumsum())
df_['n_flights_cumsum'] = df_.groupby(['FL_DATE'])['n_flights_per_hour'].transform( lambda x: x.cumsum())

In [33]:
drop_cols = ['CANCELLED', 'CANCELLATION_CODE', 'DIVERTED', 'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'LATE_AIRCRAFT_DELAY', 'Unnamed: 29']
for col in drop_cols:
    del df_[col]

In [70]:
keep = list(df_.select_dtypes(include=np.number).columns)

for col in df_.select_dtypes(exclude=np.number).columns:
    print(col, len(df_[col].unique()))
    if len(df_[col].unique()) < 100:
        keep.append(col)
        le = LabelEncoder()
        df_[col] = le.fit_transform(df_[col])
        df_[col] = df_[col].astype('category')

FL_DATE 1002
OP_UNIQUE_CARRIER 15
OP_CARRIER 15
TAIL_NUM 5363
ORIGIN 171
ORIGIN_CITY_NAME 167
ORIGIN_STATE_ABR 50
DEST 1
DEST_CITY_NAME 1
DEST_STATE_ABR 1
year_month 33
Day 274
ICAO 2


In [71]:
drop_cols = ['DEST_CITY_NAME','ORIGIN_CITY_NAME', 'Unnamed: 0']

for col in drop_cols:
    try:
        keep.remove(col)
    except:
        pass


In [72]:
df_.fillna(0, inplace=True)

In [73]:
df_.head()

,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,...,DEP_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s30_r10_median,DEP_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s30_r10_std,DEP_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s365_r10_mean,DEP_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s365_r10_std,DEP_n_flights_per_hour,ARR_n_flights_per_hour,n_flights_per_hour,DEP_n_flights_cumsum,ARR_n_flights_cumsum,n_flights_cumsum
0,2017-01-02,1,19805,1,N3CLAA,1249,12892,1289204,32575,LAX,...,0.0,0.0,0.0,0.0,52,55,107,52,55,107
1,2017-01-02,4,19790,4,N920DE,2392,11267,1126702,31267,DAY,...,0.0,0.0,0.0,0.0,85,50,135,137,105,242
2,2017-01-02,4,19790,4,N995AT,2393,12217,1221702,30255,HSV,...,0.0,0.0,0.0,0.0,85,50,135,222,155,377
3,2017-01-02,4,19790,4,N950DN,2396,14635,1463502,31714,RSW,...,0.0,0.0,0.0,0.0,73,64,137,295,219,514
4,2017-01-02,4,19790,4,N672DL,2401,13198,1319801,33198,MCI,...,0.0,0.0,0.0,0.0,85,84,169,380,303,683


In [74]:
train_df = df_.loc[(df_.FL_DATE > datetime.datetime(2018, 1, 1)) & (df_.FL_DATE <= datetime.datetime(2019, 9, 1)), keep]
test_df = df_.loc[(df_.FL_DATE > datetime.datetime(2019, 9, 1)) & (df_.FL_DATE <= datetime.datetime(2019, 9, 30)), keep]

In [75]:
print(train_df.shape)
print(test_df.shape)

(649935, 87)
(30772, 87)


In [76]:
TARGET = 'ARR_DELAY'
drop = ['DEP_DELAY']

In [77]:
features = [ col for col in train_df.columns if col != TARGET and col not in drop ]

X_train = train_df[features]
y_train = train_df[TARGET]

X_test = test_df[features]
y_test = test_df[TARGET]

In [42]:
# Negative values computed as 0
y_train[y_train < 0] = 0
y_test[y_test < 0] = 0

<ipython-input-42-5facd2a9b68d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_train[y_train < 0] = 0
<ipython-input-42-5facd2a9b68d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_test[y_test < 0] = 0


In [78]:
import lightgbm as lgbm
#reg = lgbm.LGBMRegressor(objective='tweedie', n_estimators=400)
reg = lgbm.LGBMRegressor(objective='rmse', n_estimators=400)

In [79]:
reg.fit(X_train, y_train)

LGBMRegressor(n_estimators=400, objective='rmse')

In [80]:
pd.DataFrame({'columns': X_train.columns,'feature_importance':reg.feature_importances_}).sort_values('feature_importance',ascending=False)

,columns,feature_importance
80,ORIGIN_STATE_ABR,1387
9,ARR_TIME,806
1,OP_CARRIER_FL_NUM,743
8,DEP_TIME,683
83,year_month,654
...,...,...
48,DEP_DELAY_ORIGIN_AIRPORT_ID_FL_DATE_s45_r5_mean,0
47,DEP_DELAY_ORIGIN_AIRPORT_ID_FL_DATE_s30_r5_mean,0
30,ARR_DELAY_TAIL_NUM_FL_DATE_s15_r3_mean,0
31,ARR_DELAY_TAIL_NUM_FL_DATE_s15_r5_mean,0


In [81]:
train_df_ = train_df.copy(deep=True)
train_df_['pred'] = reg.predict(X_train)

In [82]:
train_df_[['ARR_DELAY','pred']].head(20)

,ARR_DELAY,pred
364595,31.0,-9.383138
364596,-3.0,-2.384196
364597,-16.0,4.303394
364598,-16.0,4.576279
364599,-14.0,-11.385929
364600,-12.0,0.786290
364601,-17.0,5.768825
364602,23.0,20.379700
364603,-1.0,-4.815594
364604,-1.0,-7.738607


In [83]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(train_df_['ARR_DELAY'], reg.predict(X_train), squared=True)
rmse

1322.4260920463712

In [84]:
rmse = mean_squared_error(test_df['ARR_DELAY'], reg.predict(X_test), squared=True)
rmse

1042.2043028473236

In [50]:
train_df_[['ARR_DELAY','pred']].sort_values('ARR_DELAY', ascending=False).head(20)

,ARR_DELAY,pred
731142,2108.0,1053.486040
712110,1484.0,43.651917
807047,1464.0,108.439380
425696,1400.0,143.376601
757529,1390.0,50.283101
686062,1370.0,38.880878
386144,1366.0,54.601954
960356,1365.0,167.071241
861543,1364.0,104.842737
695980,1359.0,44.525583


In [51]:
train_df_[['ARR_DELAY','pred']].sort_values('ARR_DELAY', ascending=False).head(20)

,ARR_DELAY,pred
731142,2108.0,1053.486040
712110,1484.0,43.651917
807047,1464.0,108.439380
425696,1400.0,143.376601
757529,1390.0,50.283101
686062,1370.0,38.880878
386144,1366.0,54.601954
960356,1365.0,167.071241
861543,1364.0,104.842737
695980,1359.0,44.525583


# Classication

In [85]:
y_train_ = y_train > 15
y_test_ = y_test > 15

In [88]:
import lightgbm as lgbm
#reg = lgbm.LGBMRegressor(objective='tweedie', n_estimators=400)
reg = lgbm.LGBMClassifier(metric='roc_auc', n_estimators=400)

In [89]:
reg.fit(X_train, y_train_)

LGBMClassifier(metric='roc_auc', n_estimators=400)

In [90]:
pd.DataFrame({'columns': X_train.columns,'feature_importance':reg.feature_importances_}).sort_values('feature_importance',ascending=False)

,columns,feature_importance
80,ORIGIN_STATE_ABR,1905
9,ARR_TIME,1226
83,year_month,971
1,OP_CARRIER_FL_NUM,903
8,DEP_TIME,863
...,...,...
53,DEP_DELAY_ORIGIN_AIRPORT_ID_FL_DATE_s30_r30_mean,0
54,DEP_DELAY_ORIGIN_AIRPORT_ID_FL_DATE_s45_r30_mean,0
30,ARR_DELAY_TAIL_NUM_FL_DATE_s15_r3_mean,0
31,ARR_DELAY_TAIL_NUM_FL_DATE_s15_r5_mean,0


In [91]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_train_, reg.predict(X_train))

array([[547352,   5727],
       [ 64592,  32264]])

In [52]:
# Feature Generation

In [93]:
y_train_.value_counts()

False    553079
True      96856
Name: ARR_DELAY, dtype: int64